# ML.Net - SeedClustering

In [5]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 

## Using C# Class

In [10]:
using System;
using Microsoft.ML;
using Microsoft.ML.Data;
using static Microsoft.ML.DataOperationsCatalog;

## Declare data-classes for input data and predictions

In [7]:
public class SeedData
{
    public float A;
    public float P;
    public float C;
    public float LK;
    public float WK;
    public float A_Coef;
    public float LKG;
    public float Label;
}

public class SeedPrediction
{
    [ColumnName("PredictedLabel")]
    public uint SelectedClusterId;
    [ColumnName("Score")]
    public float[] Distance;
}

## Evaluate

In [12]:
var dataLocation = "./datasets/Seed_data/Seed_Data.csv";

var context = new MLContext();

var textLoader = context.Data.CreateTextLoader(new[]
{
    new TextLoader.Column("A", DataKind.Single, 0),
    new TextLoader.Column("P", DataKind.Single, 1),
    new TextLoader.Column("C", DataKind.Single, 2),
    new TextLoader.Column("LK", DataKind.Single, 3),
    new TextLoader.Column("WK", DataKind.Single, 4),
    new TextLoader.Column("A_Coef", DataKind.Single, 5),
    new TextLoader.Column("LKG", DataKind.Single, 6),
    new TextLoader.Column("Label", DataKind.Single, 7)
},
hasHeader: true,
separatorChar: ',');


var data = context.Data.LoadFromTextFile<SeedData>("./datasets/Seed_data/Seed_Data.csv", hasHeader: true, separatorChar: ',', allowQuoting: true);


IDataView data = textLoader.Load(dataLocation);

var trainTestData = context.Clustering.TrainTestSplit(data, testFraction: 0.2);

var pipeline = context.Transforms.Concatenate("Features", "A", "P", "C", "LK", "WK", "A_Coef", "LKG")
    .Append(context.Clustering.Trainers.KMeans(featureColumnName: "Features", clustersCount: 3));

var preview = trainTestData.TrainSet.Preview();

var model = pipeline.Fit(trainTestData.TrainSet);

var predictions = model.Transform(trainTestData.TestSet);

var metrics = context.Clustering.Evaluate(predictions, score: "Score", features: "Features");

Console.WriteLine($"Average minimum score: {metrics.AvgMinScore}");

var predictionFunc = model.CreatePredictionEngine<SeedData, SeedPrediction>(context);

var prediction = predictionFunc.Predict(new SeedData
{
    A = 13.89F,
    P = 15.33F,
    C = 0.862F,
    LK = 5.42F,
    WK = 3.311F,
    A_Coef = 2.8F,
    LKG = 5
});

Console.WriteLine($"Prediction - {prediction.SelectedClusterId}");
Console.ReadLine();

Unhandled exception: (7,31): error CS0120: Uma referência de objeto é necessária para o campo, o método ou a propriedade "MLContext.Data" não estática